<a href="https://colab.research.google.com/github/ajitpatil102001/Final-Project/blob/main/Samsung_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Text Based Mood Profiling:Emotion Inteligence**

In [2]:
import nltk #Natural lang toolkit
nltk.download('stopwords')

import pandas as pd# data analysis
import seaborn as sns #Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical
import numpy as np#NumPy is a Python library used for working with arrays. It also has functions for working in domain of linear algebra, fourier transform, and matrices.
import xgboost as xgb#XGBoost dominates structured or tabular datasets on classification and regression predictive modeling problems
from tqdm import tqdm# tqdm is a library in Python which is used for creating Progress Meters or Progress Bars.
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding

from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
train = pd.read_csv('/content/sample_data/hello.csv')
cv = pd.read_csv('/content/sample_data/hello.csv')
test = pd.read_csv('/content/sample_data/hello.csv')

In [ ]:
train.shape, cv.shape , test.shape

((9989, 4), (9989, 4), (9989, 4))

In [ ]:
test.columns


Index(['Statements', 'Emotion', 'StartTime', 'EndTime'], dtype='object')

In [ ]:
train.columns

Index(['Statements', 'Emotion', 'StartTime', 'EndTime'], dtype='object')

In [ ]:
cv.columns

Index(['Statements', 'Emotion', 'StartTime', 'EndTime'], dtype='object')

In [5]:
train.head()

,Statements,Emotion,StartTime,EndTime
0,also I was the point person on my companys tr...,neutral,"00:16:16,059","00:16:21,731"
1,You mustve had your hands full.,neutral,"00:16:21,940","00:16:23,442"
2,That I did. That I did.,neutral,"00:16:23,442","00:16:26,389"
3,So lets talk a little bit about your duties.,neutral,"00:16:26,820","00:16:29,572"
4,My duties? All right.,surprise,"00:16:34,452","00:16:40,917"


In [ ]:
ax = sns.countplot(x=train['Emotion'], data=train)
print(train['Emotion'].value_counts())

neutral     4709
joy         1743
surprise    1205
anger       1109
sadness      684
disgust      271
fear         268
Name: Emotion, dtype: int64


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9989 entries, 0 to 9988
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Statements  9989 non-null   object
 1   Emotion     9989 non-null   object
 2   StartTime   9989 non-null   object
 3   EndTime     9989 non-null   object
dtypes: object(4)
memory usage: 312.3+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9989 entries, 0 to 9988
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Statements  9989 non-null   object
 1   Emotion     9989 non-null   object
 2   StartTime   9989 non-null   object
 3   EndTime     9989 non-null   object
dtypes: object(4)
memory usage: 312.3+ KB


In [ ]:
X_train , y_train = train[['Statements']] , train[['Emotion']]
X_test , y_test = test[['Statements']] , test[['Emotion']]

In [ ]:
lbl_enc = preprocessing.LabelEncoder()
y_train_enc = lbl_enc.fit_transform(y_train.Emotion.values)
y_test_enc = lbl_enc.transform(y_test.Emotion.values)

In [ ]:
y_train.Emotion.values[1:100]

array(['neutral', 'neutral', 'neutral', 'surprise', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'fear', 'neutral', 'surprise',
       'neutral', 'surprise', 'sadness', 'surprise', 'fear', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'joy', 'sadness',
       'surprise', 'neutral', 'disgust', 'sadness', 'neutral', 'neutral',
       'joy', 'neutral', 'joy', 'surprise', 'surprise', 'surprise',
       'neutral', 'neutral', 'neutral', 'surprise', 'sadness', 'neutral',
       'surprise', 'joy', 'surprise', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'joy', 'joy', 'joy', 'sadness', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'surprise',
       'joy', 'surprise', 'joy', 'neutral', 'neutral', 'anger', 'joy',
       'neutral', 'surprise', 'anger', 'anger', 'anger', 'neutral',
       'neutral', 'sadness', 'sadness', 'sadness', 'surprise', 'anger',
       'anger', 'anger', 'anger', 'neutral', 'anger', 'neutral',
     

In [ ]:
import re

### Dataset Preprocessing training set
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
train_corpus = []
for i in range(0, len(X_train)):
    review = re.sub('[^a-zA-Z]', ' ', X_train['Statements'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    train_corpus.append(review)

In [ ]:
import re

### Dataset Preprocessing test set
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
test_corpus = []
for i in range(0, len(X_test)):
    review = re.sub('[^a-zA-Z]', ' ', X_test['Statements'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    test_corpus.append(review)